## Imports

In [1]:
#General imports for main#
#------------------------#

#Main libraries imports
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy as sp
print('scipy: {}'.format(sp.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
# keras
import keras
print('keras: {}'.format(keras.__version__))

Python: 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]


scipy: 1.0.0
numpy: 1.14.0


pandas: 0.22.0


sklearn: 0.19.1


D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


keras: 2.2.0


In [ ]:
# ML alogorithems, model selection, feature selection

#Algorithems
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Model selection 
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, cross_validate

# Feature selection
from sklearn.feature_selection import RFECV

In [ ]:
# ML alogorithems, model selection, feature selection   

#Keras layers and optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam

In [3]:
# Imports for saving and file formating 

from os import walk
import re


## Function for saving

In [ ]:

#Define an iterative file name for scv files created
def savetofile(dframe, score_dict, dir):

    #Get a
    f = []
    for (dirpath, dirnames, filenames) in walk(dir):
        f.extend(filenames)
        break

    for filename in f:
        number = 0
        match = re.match("predictions_(\d+).*", filename)
        if match == None:
            continue
        if int(match.group(1)) > number:
            number = match.group(1)
    number = int(number) + 1

    filename = "predictions"
    number = number.__str__()
    date = datetime.date.today().strftime("%d-%m-%y")
    score = np.round(score_dict["NN"],2).__str__()
    suffix = "csv"

    str = '__'.join([filename, number, date, score])
    path = '/'.join([dir, str])
    path = '.'.join([path, suffix])

    dframe.to_csv(path, index=False)


## Function for file log 

In [4]:


def logfile(create_model,cross_val_score_results, fetures ):
    f = "predictions/log.txt"
    a = create_model()
    loss = str(a.loss)
    opt=str(a.optimizer.__class__)
    print_fn = lambda x: handle.write(x + '\n')

    with open(f, "a") as handle:
        a.summary(print_fn=print_fn)
        handle.write("loss= "+loss+",opt= "+opt+"\n")
        handle.write("opt= "+opt+"\n")
        handle.write("accuracy: " + str(cross_val_score_results)+"\n\n")
        handle.write("fetures "+str(fetures)+"\n")

## NN init 

In [5]:
def create_model(optimizer=RMSprop,lr=0.0001, loss= 'binary_crossentropy', input_dim= 21,
                 layer_a=30, layer_b=30,layer_c=12,
                 activation_a = 'relu' ,activation_b ='relu',activation_c ='sigmoid',):

    # create model
    model = Sequential()
    model.add(Dense(layer_a, input_dim=input_dim, activation=activation_a))
    model.add(Dense(layer_b, activation=activation_b))
    model.add(Dense(layer_c, activation=activation_c))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    #optimzer = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss=loss, optimizer=optimizer(lr=lr), metrics=['accuracy'])
    return model

## Main code

### Data preperation 

In [6]:
#Load df from file
data1 = pd.read_csv("train.csv")
data2 = pd.read_csv("test.csv")

sets = [data1,data2]

In [7]:
#Deal with NA values
for set in sets:
    set['Age'].fillna(set['Age'].median(), inplace=True)
    set['Embarked'].fillna(set['Embarked'].mode()[0], inplace = True)
    set['Fare'].fillna(set['Fare'].median(), inplace = True)
    set.drop("Cabin", inplace =True, axis = 1)

In [8]:
#Make a Title column
for set in sets:
    set["Title"]= set.Name.str.extract('^.*?,\s(.*?)\.', expand = False)
    #CLean Title
    validTitles = (set["Title"].value_counts()/set["Title"].shape[0])<0.1
    set["Title"] = set["Title"].apply(lambda x: "Misc" if validTitles.loc[x] == True else x )

In [9]:
#Binning relevent columns
for set in sets:
    set['AgeBin'] = pd.cut(set['Age'].astype(int), 5) #  pd.cut(set['Age'].astype(int), 4, labels = (0,1,2,3), retbins=True)[1]  ;;    data1['AgeBin'].unique()
    set['FareBin'] = pd.qcut(set['Fare'], 4) # pd.qcut(set['Fare'], 4, labels =(0,1,2,3), retbins = True)[1]

    set['FareBin'].dtypes.categories.values.astype(str)


In [10]:
# Data reformatting 

#Y column
Target = ['Survived']

#One hot data
dummy_sets = []
for set in sets:
    dummy = pd.get_dummies(set[['AgeBin', 'FareBin', 'Sex', 'Embarked', 'Title',"Pclass","SibSp","Parch"]])
    dummy = dummy.astype(int)
    dummy_sets.append(dummy)

dummy1 = dummy_sets[0]
dummy2 = dummy_sets[1]

In [ ]:
#Split to train and test
X_train, X_test, y_train, y_test =train_test_split(dummy1.values, data1[Target])
#Dummy y values
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
X= dummy1.values
Y= keras.utils.to_categorical(data1[Target])

### Build NN, cross validate with hyperparameter selection 

In [12]:
# Create a sklearn keras model 
model_w_Wrapper = KerasClassifier(build_fn=create_model, epochs=300, batch_size = 500,verbose=0)

In [ ]:
# define the grid search parameters

#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#optimizer = [SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam]
optimizer = [SGD, RMSprop]
#loss = ["mean_squared_error", "mean_absolute_error", "mean_absolute_percentage_error",\
#        "mean_squared_logarithmic_error", "squared_hinge","hinge","categorical_hinge",\
#        "logcosh","categorical_crossentropy", "sparse_categorical_crossentropy",\
#        "binary_crossentropy", "kullback_leibler_divergence", "poisson", "cosine_proximity"]
loss = ["mean_squared_error","binary_crossentropy"]
layer_a= layer_b = layer_c = np.arange(10,40, step=10)
activation_a = activation_b = activation_c = ["relu", "sigmoid"]
lr = np.linspace(0.000001,0.0001,4)
momentum = np.linspace(0.01 ,0.0001,4)

In [ ]:
# make a parameter grid 

#param_grid = dict(optimizer=optimizer, loss= loss, layer_a =layer_a, layer_b =layer_b, layer_c=layer_c,
#                  activation_a=activation_a, activation_b=activation_b, activation_c=activation_c,
#                  lr=lr,momentum=momentum)

param_grid = dict(optimizer=optimizer, loss= loss)

In [17]:
# Cross validate with hyperparameter selction  

grid = GridSearchCV(estimator=model_w_Wrapper, param_grid=param_grid, verbose=3)
grid_result = grid.fit(X_train, y_train)

grid_result.cv_results_['mean_train_score'][grid_result.best_index_]
grid_result.cv_results_['params'][grid_result.best_index_]

Epoch 1/300


445/445 [==============================] - 0s 577us/step - loss: 0.3110 - acc: 0.3775


Epoch 2/300
445/445 [==============================] - 0s 5us/step - loss: 0.3110 - acc: 0.3775


Epoch 3/300
445/445 [==============================] - 0s 4us/step - loss: 0.3110 - acc: 0.3775


Epoch 4/300
445/445 [==============================] - 0s 9us/step - loss: 0.3110 - acc: 0.3775


...

Epoch 102/300
445/445 [==============================] - 0s 4us/step - loss: 0.2732 - acc: 0.3506


Epoch 103/300
445/445 [==============================] - 0s 4us/step - loss: 0.2728 - acc: 0.3506


Epoch 104/300
445/445 [==============================] - 0s 5us/step - loss: 0.2725 - acc: 0.3506


Epoch 105/300


445/445 [==============================] - 0s 5us/step - loss: 0.2722 - acc: 0.3528
